In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from akvmodel import *
from pgml import *

In [2]:
def draw_graph(model: Model):
    DG = nx.DiGraph()
    world_size = model.worlds_size
    DG.add_nodes_from(list(range(model.worlds_size)))
    DG.add_weighted_edges_from(
        [
            (i, j, model.relation[i][j])
            for i in range(world_size)
            for j in range(world_size)
            if model.relation[i][j] != 0
        ]
    )
    # labels = nx.get_edge_attributes(G, "weights")
    nx.draw(DG)

In [3]:
# F(p)
class Future(Expression):
    def __init__(self, operand: Expression):
        self.operand = operand

    def valuation1(self, models: List[Model], world: int) -> float:
        return max(self.operand.valuation1(model, world) for model in models)

In [4]:
left = Box(Variable("p"))
right = Variable("p")
impl = Implication(left, right)

In [5]:
def build_pgml_models_from_akv(akv_model):
    # Build relation
    relation = [
        [akv_model.influence_graph[j][i] for j in range(NUM_AGENTS)] for i in range(NUM_AGENTS)
    ]
    # Remove self-loops
    for i in range(10):
        relation[i][i] = 0

    # Create a PGML model for each step
    pgml_models = []
    for i in range(NUM_STEPS + 1):
        pgml_models += [
            Model(
                NUM_AGENTS,
                relation,
                {"p": akv_model.states[i][0].tolist()},
                {"p": akv_model.states[i][1].tolist()},
            )
        ]
    
    return pgml_models

# Example 1: Cique, mildly polarized, classic update

In [6]:
# Create AKV Model
NUM_AGENTS = 10
NUM_STEPS = 10
akv_model = AKV(
    belief_state=InitialConfigurations.mildly(NUM_AGENTS),
    influence_graph=InfluenceGraphs.clique(NUM_AGENTS),
    update_function=UpdateFunctions.classic
)

# Run AKV 100 times
for i in range(NUM_STEPS):
    akv_model.update()

pgml_models = build_pgml_models_from_akv(akv_model)

In [7]:
n = NUM_AGENTS
table = [
    {
        "agent 0 valuation 1": akv_model.states[t][0][0],
        "[]p": left.valuation1(pgml_models[t], 0),
        "p": right.valuation1(pgml_models[t], 0),
        "[]p -> p": impl.valuation1(pgml_models[t], 0),
    }
    for t in range(NUM_STEPS + 1)
]
pd.DataFrame(table)

,agent 0 valuation 1,[]p,p,[]p -> p
0,0.20,0.24,0.20,0.2
1,0.48,0.48,0.48,1.0
2,0.48,0.48,0.48,1.0
3,0.48,0.48,0.48,1.0
4,0.48,0.48,0.48,1.0
5,0.48,0.48,0.48,1.0
6,0.48,0.48,0.48,1.0
7,0.48,0.48,0.48,1.0
8,0.48,0.48,0.48,1.0
9,0.48,0.48,0.48,1.0


In [8]:
# F [] p
Future(left).valuation1(pgml_models, 0)

0.48

In [9]:
# F p
Future(right).valuation1(pgml_models, 0)

0.48000000000000004

In [10]:
# F([] p -> p)
Future(impl).valuation1(pgml_models, 0)

1

In [11]:
# F [] p -> F p
Implication(Future(left), Future(right)).valuation1(pgml_models, 0)

1

# Example 2: Faintly connected, mildly polarized, classic

In [12]:
# Create AKV Model
NUM_AGENTS = 10
NUM_STEPS = 100
akv_model = AKV(
    belief_state=InitialConfigurations.mildly(NUM_AGENTS),
    influence_graph=InfluenceGraphs.faintly(NUM_AGENTS),
    update_function=UpdateFunctions.classic
)

# Run AKV 100 times
for i in range(NUM_STEPS):
    akv_model.update()

pgml_models = build_pgml_models_from_akv(akv_model)

In [13]:
n = NUM_AGENTS
table = [
    {
        "agent 0 valuation 1": akv_model.states[t][0][0],
        "[]p": left.valuation1(pgml_models[t], 0),
        "p": right.valuation1(pgml_models[t], 0),
        "[]p -> p": impl.valuation1(pgml_models[t], 0),
    }
    for t in range(NUM_STEPS + 1)
]
pd.DataFrame(table)

,agent 0 valuation 1,[]p,p,[]p -> p
0,0.200000,0.240000,0.200000,0.200000
1,0.244000,0.272000,0.244000,0.244000
2,0.278800,0.298400,0.278800,0.278800
3,0.306760,0.320480,0.306760,0.306760
4,0.329572,0.339176,0.329572,0.329572
...,...,...,...,...
96,0.479992,0.479992,0.479992,0.479992
97,0.479993,0.479993,0.479993,0.479993
98,0.479993,0.479993,0.479993,1.000000
99,0.479994,0.479994,0.479994,1.000000


In [14]:
# F [] p
Future(left).valuation1(pgml_models, 0)

0.47999468772022247

In [15]:
# F p
Future(right).valuation1(pgml_models, 0)

0.47999468772022247

In [16]:
# F([] p -> p)
Future(impl).valuation1(pgml_models, 0)

1

In [17]:
# F [] p -> F p
Implication(Future(left), Future(right)).valuation1(pgml_models, 0)

1

# Example 3: Disconnected, mildly polarized, classic

In [18]:
# Create AKV Model
NUM_AGENTS = 10
NUM_STEPS = 100
akv_model = AKV(
    belief_state=InitialConfigurations.mildly(NUM_AGENTS),
    influence_graph=InfluenceGraphs.disconnected(NUM_AGENTS),
    update_function=UpdateFunctions.classic
)

# Run AKV 100 times
for i in range(NUM_STEPS):
    akv_model.update()

pgml_models = build_pgml_models_from_akv(akv_model)

In [19]:
n = NUM_AGENTS
table = [
    {
        "agent 0 valuation 1": akv_model.states[t][0][0],
        "[]p": left.valuation1(pgml_models[t], 0),
        "p": right.valuation1(pgml_models[t], 0),
        "[]p -> p": impl.valuation1(pgml_models[t], 0),
    }
    for t in range(NUM_STEPS + 1)
]
pd.DataFrame(table)

,agent 0 valuation 1,[]p,p,[]p -> p
0,0.200,0.2400,0.200,0.200
1,0.240,0.2600,0.240,0.240
2,0.260,0.2700,0.260,0.260
3,0.270,0.2750,0.270,0.270
4,0.275,0.2775,0.275,0.275
...,...,...,...,...
96,0.280,0.2800,0.280,1.000
97,0.280,0.2800,0.280,1.000
98,0.280,0.2800,0.280,1.000
99,0.280,0.2800,0.280,1.000


In [20]:
# F [] p
Future(left).valuation1(pgml_models, 0)

0.27999999999999997

In [21]:
# F p
Future(right).valuation1(pgml_models, 0)

0.27999999999999997

In [22]:
# F([] p -> p)
Future(impl).valuation1(pgml_models, 0)

1

In [23]:
# F [] p -> F p
Implication(Future(left), Future(right)).valuation1(pgml_models, 0)

1